# AtomsBase integration

[AtomsBase.jl](https://github.com/JuliaMolSim/AtomsBase.jl) is a common interface
for representing atomic structures in Julia. DFTK directly supports using such
structures to run a calculation as is demonstrated here.

In [1]:
using DFTK

## Feeding an AtomsBase AbstractSystem to DFTK
In this example we construct a silicon system using the `ase.build.bulk` routine
from the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
(ASE), which is exposed by [ASEconvert](https://github.com/mfherbst/ASEconvert.jl)
as an AtomsBase `AbstractSystem`.

In [2]:
# Construct bulk system and convert to an AbstractSystem
using ASEconvert
system_ase = ase.build.bulk("Si")
system = pyconvert(AbstractSystem, system_ase)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0    2.715    2.715;
                            2.715        0    2.715;
                            2.715    2.715        0]u"Å"

    Atom(Si, [       0,        0,        0]u"Å")
    Atom(Si, [  1.3575,   1.3575,   1.3575]u"Å")

                       
                       
                       
                       
              Si       
                       
          Si           
                       
                       
                       
                       


To use an AbstractSystem in DFTK, we attach pseudopotentials, construct a DFT model,
discretise and solve:

In [3]:
system = attach_psp(system; Si="hgh/lda/si-q4")

model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921736670174                   -0.69    6.0    346ms
  2   -7.926164330074       -2.35       -1.22    1.0    171ms
  3   -7.926838660331       -3.17       -2.37    1.9    202ms
  4   -7.926861190772       -4.65       -3.03    2.2    279ms
  5   -7.926861645070       -6.34       -3.38    1.8    195ms
  6   -7.926861668133       -7.64       -3.75    1.6    175ms
  7   -7.926861679059       -7.96       -4.11    1.5    171ms
  8   -7.926861681796       -8.56       -5.11    1.5    175ms
  9   -7.926861681862      -10.19       -5.21    2.8    255ms
 10   -7.926861681872      -10.98       -6.45    1.0    163ms
 11   -7.926861681873      -12.30       -6.64    3.5    261ms
 12   -7.926861681873      -15.05       -7.35    1.0    167ms
 13   -7.926861681873      -14.45       -7.81    2.1    196ms
┌ Warning: Eigensolver not converged
│   n_iter =
│    8-element Vector

If we did not want to use ASE we could of course use any other package
which yields an AbstractSystem object. This includes:

### Reading a system using AtomsIO

In [4]:
using AtomsIO

# Read a file using [AtomsIO](https://github.com/mfherbst/AtomsIO.jl),
# which directly yields an AbstractSystem.
system = load_system("Si.extxyz")

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921727494782                   -0.69    6.0    326ms
  2   -7.926163173364       -2.35       -1.22    1.0    193ms
  3   -7.926838221043       -3.17       -2.37    1.9    206ms
  4   -7.926861182958       -4.64       -3.02    2.2    269ms
  5   -7.926861639461       -6.34       -3.35    2.0    210ms
  6   -7.926861665698       -7.58       -3.71    1.6    176ms
  7   -7.926861679449       -7.86       -4.16    1.0    163ms
  8   -7.926861681805       -8.63       -5.18    1.8    193ms
  9   -7.926861681862      -10.25       -5.22    3.0    264ms
 10   -7.926861681872      -10.99       -6.47    1.0    166ms
 11   -7.926861681873      -12.33       -6.72    3.1    259ms
 12   -7.926861681873      -14.10       -7.18    1.1    169ms
 13   -7.926861681873   +  -15.05       -7.91    1.6    176ms
┌ Warning: Eigensolver not converged
│   n_iter =
│    8-element Vector

The same could be achieved using [ExtXYZ](https://github.com/libAtoms/ExtXYZ.jl)
by `system = Atoms(read_frame("Si.extxyz"))`,
since the `ExtXYZ.Atoms` object is directly AtomsBase-compatible.

### Directly setting up a system in AtomsBase

In [5]:
using AtomsBase
using Unitful
using UnitfulAtomic

# Construct a system in the AtomsBase world
a = 10.26u"bohr"  # Silicon lattice constant
lattice = a / 2 * [[0, 1, 1.],  # Lattice as vector of vectors
                   [1, 0, 1.],
                   [1, 1, 0.]]
atoms  = [:Si => ones(3)/8, :Si => -ones(3)/8]
system = periodic_system(atoms, lattice; fractional=true)

# Now run the LDA calculation:
system = attach_psp(system; Si="hgh/lda/si-q4")
model  = model_LDA(system; temperature=1e-3)
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.921715227614                   -0.69    6.0    403ms
  2   -7.926166364200       -2.35       -1.22    1.0    180ms
  3   -7.926841757470       -3.17       -2.37    1.9    235ms
  4   -7.926864592758       -4.64       -3.02    2.2    253ms
  5   -7.926865048601       -6.34       -3.35    2.1    232ms
  6   -7.926865075090       -7.58       -3.68    1.5    181ms
  7   -7.926865090565       -7.81       -4.16    1.1    167ms


## Obtaining an AbstractSystem from DFTK data

At any point we can also get back the DFTK model as an
AtomsBase-compatible `AbstractSystem`:

In [6]:
second_system = atomic_system(model)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0     5.13     5.13;
                             5.13        0     5.13;
                             5.13     5.13        0]u"a₀"

    Atom(Si, [  1.2825,   1.2825,   1.2825]u"a₀")
    Atom(Si, [ -1.2825,  -1.2825,  -1.2825]u"a₀")

                       
                       
                       
                       
              Si       
                       
          Si           
                       
                       
                       
                       


Similarly DFTK offers a method to the `atomic_system` and `periodic_system` functions
(from AtomsBase), which enable a seamless conversion of the usual data structures for
setting up DFTK calculations into an `AbstractSystem`:

In [7]:
lattice = 5.431u"Å" / 2 * [[0 1 1.];
                           [1 0 1.];
                           [1 1 0.]];
Si = ElementPsp(:Si; psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

third_system = atomic_system(lattice, atoms, positions)

FlexibleSystem(Si₂, periodic = TTT):
    bounding_box      : [       0  5.13155  5.13155;
                          5.13155        0  5.13155;
                          5.13155  5.13155        0]u"a₀"

    Atom(Si, [ 1.28289,  1.28289,  1.28289]u"a₀")
    Atom(Si, [-1.28289, -1.28289, -1.28289]u"a₀")

                       
                       
                       
                       
              Si       
                       
          Si           
                       
                       
                       
                       
